# Lab 2b: Eigendecomposition of Stoichiometric Matrices
In this lab, we'll explore the eigendecomposition of a standard matrix in chemical systems: the stoichiometric matrix $\mathbf{S}$. The stoichiometric matrix is the digital representation of a chemical reaction system.

### What is a stoichiometric matrix?

Suppose we have a set of chemical (or biochemical) reactions $\mathcal{R}$ involving the chemical species (metabolite) set $\mathcal{M}$. Then, the stoichiometric matrix is a $\mathbf{S}\in\mathbb{R}^{|\mathcal{M}|\times|\mathcal{R}|}$ matrix that holds the stoichiometric coefficients $\sigma_{ij}\in\mathbf{S}$ such that:
* $\sigma_{ij}>0$: Chemical species (metabolite) $i$ is _produced_ by reaction $j$. Species $i$ is a product of reaction $j$.
* $\sigma_{ij} = 0$: Chemical species (metabolite) $i$ is not connected with reaction $j$
* $\sigma_{ij}<0$: Chemical species (metabolite) $i$ is _consumed_ by reaction $j$. Species $i$ is a reactant of reaction $j$.


### Tasks

Before we start, divide into teams and familiarize yourself with the lab. Then, execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!

* __Task 1: Setup, Data, Constants (20 min)__: Let's take 20 minutes to review the dataset we'll explore today and set up some values we'll use in the other tasks. We'll load the data and do some initial _data munging_ (also called [data wrangling](https://en.wikipedia.org/wiki/Data_wrangling)) to get the dataset in a form that we'll use in our analysis.
* __Task 2: Compute the eigendecomposition (10 min)__: In this task, we'll compute the eigendecomposition of the $\mathbf{\Sigma}$ matrix [using the `eigen(...)` method exported by the `LinearAlgebra.jl` package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) and some checks on the results.
* __Task 3: What does the largest eigenvalue/eigenvector tell us (20 min)?__: Now that we have the eigenvalues and eigenvectors, in this task, we'll explore what the largest eigenvector coefficients tell us about the $\mathbf{\Sigma}$ matrix and, by extension, the stoichiometric matrix $\mathbf{S}$. Is there something interesting here?

## Task 1: Setup, Data and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl"); # load packages, and sets up the environment

  Activating project at `~/Documents/GitHub/CHEME-5820-Labs-Spring-2025/labs/week-2/L2b`
   Installed CompositionsBase ─ v0.1.2
   Installed ZygoteRules ────── v0.2.7
   Installed Functors ───────── v0.4.12
   Installed KernelFunctions ── v0.10.64
    Updating `~/Documents/GitHub/CHEME-5820-Labs-Spring-2025/labs/week-2/L2b/Project.toml`
  [336ed68f] + CSV v0.10.15
  [a93c6f00] + DataFrames v1.7.0
  [5789e2e9] + FileIO v1.16.6
  [cd3eb016] + HTTP v1.10.15
  [033835bb] + JLD2 v0.5.11
  [682c06a0] + JSON v0.21.4
  [ec8451be] + KernelFunctions v0.10.64
  [10745b16] + Statistics v1.11.1
  [b27032c2] ~ LibCURL ⇒ v0.6.4
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Documents/GitHub/CHEME-5820-Labs-Spring-2025/labs/week-2/L2b/Manifest.toml`
  [66dad0bd] + AliasTables v1.1.3
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [d360d2e6] + ChainRulesCore v1.25.1
  [944b1d66] + CodecZlib v0.7.6
  [34da2185] + Compat v4.16.0
  [a33af91c] + CompositionsBase v0.1.2
  [f0e56b4a] + C

### Data
We developed a simple software development kit (SDK) against [the BiGG Models application programming interface at the University of California, San Diego](http://bigg.ucsd.edu/). The [BiGG Models database](http://bigg.ucsd.edu/) integrates published genome-scale metabolic networks into a single database with standardized nomenclature and structure. 
* [The BiGG models API](http://bigg.ucsd.edu/data_access) allows users to programmatically access genome-scale stoichiometric model reconstructions using a simple web API. There are `108` models of intracellular biochemistry occurring in various organisms (including humans) in the database (so far); [see here for a list of models](http://bigg.ucsd.edu/models).
* Here, we'll first explore the [core metabolic model of Palsson and coworkers](https://pubmed.ncbi.nlm.nih.gov/26443778/), which is a scaled-down model of [carbohydrate metabolism](https://en.wikipedia.org/wiki/Carbohydrate_metabolism) in _E.coli_. This model has 72 metabolites and 95 reactions. We'll then look at other models, and see what is going on with these.

We call the model download endpoint of [the BiGG models API](http://bigg.ucsd.edu/data_access) and then save the model file to disk (so we don't hit the API unless we have to). This call returns model information organized as [a Julia dictionary](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) in the `model::Dict{String, Any}` variable. If a model file is saved, we use the cached file instead of making an API call.

In [2]:
model = let

    # build download endpoint -
    baseurl = "http://bigg.ucsd.edu"; # base url to download model
    modelid = "iAB_RBC_283"; # model id to download
    path_to_saved_model_file = joinpath(_PATH_TO_DATA, "saved-model-$(modelid).jld2");

    # check: do we have a model file saved?
    model = nothing;
    if (isfile(path_to_saved_model_file) == false)
        
        endpoint = MyBiggModelsDownloadModelEndpointModel();
        endpoint.bigg_id = modelid;
        url = build(baseurl, endpoint)
        model = MyBiggModelsDownloadModelEndpointModel(url);

        # Before we move on, save this model for later (so we don't keep hitting the API)
        save(path_to_saved_model_file, Dict("model" => model));
    else
        model = load(path_to_saved_model_file)["model"];
    end
    model; # return the model (either saved, or downloaded)
end

Dict{String, Any} with 6 entries:
  "metabolites"  => Any[Dict{String, Any}("compartment"=>"c", "name"=>"3-Phosph…
  "id"           => "iAB_RBC_283"
  "compartments" => Dict{String, Any}("c"=>"cytosol", "e"=>"extracellular space…
  "reactions"    => Any[Dict{String, Any}("name"=>"Sink pchol hs 18 1 18 1(c)",…
  "version"      => "1"
  "genes"        => Any[Dict{String, Any}("name"=>"NMRK1", "id"=>"Nrk1_AT1", "n…

__Metabolite records__: Each metabolite (chemical compound) in the network has an associated metabolite record with several fields. Let's take a look at the metabolite at index `1`.
* The key field for today in the metabolite record is the `id` field, an abbreviation or symbol associated with this metabolite.

In [3]:
model["metabolites"][1] # example metabolite record

Dict{String, Any} with 7 entries:
  "compartment" => "c"
  "name"        => "3-Phospho-D-glyceroyl phosphate"
  "formula"     => "C3H4O10P2"
  "id"          => "13dpg_c"
  "charge"      => -4
  "notes"       => Dict{String, Any}("original_bigg_ids"=>Any["13dpg_c"])
  "annotation"  => Dict{String, Any}("sabiork"=>Any["21215"], "kegg.compound"=>…

__Reaction records__: Similarly, each reaction in the network has a reaction record with several fields. Let's look at the reaction record at index `25`.
* The key field for the reaction record is the `metabolites` field, which lists the stoichiometric coefficients associated with this particular reaction.

In [4]:
model["reactions"][25] # example reaction record

Dict{String, Any} with 9 entries:
  "name"               => "N-Acetylneuraminate 9-phosphate pyruvate-lyase (pyru…
  "metabolites"        => Dict{String, Any}("h2o_c"=>-1.0, "acnamp_c"=>1.0, "ac…
  "lower_bound"        => 0.0
  "id"                 => "ACNAM9PL"
  "notes"              => Dict{String, Any}("original_bigg_ids"=>Any["ACNAM9PL"…
  "gene_reaction_rule" => "Nans_AT1"
  "upper_bound"        => 1000.0
  "subsystem"          => "Aminosugar Metabolism"
  "annotation"         => Dict{String, Any}("bigg.reaction"=>Any["ACNAM9PL"], "…

In [5]:
model["reactions"][25]["metabolites"]

Dict{String, Any} with 5 entries:
  "h2o_c"     => -1.0
  "acnamp_c"  => 1.0
  "acmanap_c" => -1.0
  "pep_c"     => -1.0
  "pi_c"      => 1.0

__Stoichiometric matrix__: Next, let's build a stoichiometric matrix $\mathbf{S}$ using the metabolite and reaction records. We'll do this using two for loops. 
* __Strategy__: In the outer loop, we iterate over the system's metabolites (chemical species) and select the `id` from the metabolites record for each metabolite. In the inner loop, we iterate over each reaction. For each reaction record, we ask if this reaction has an entry for the current metabolite `id` value; if it does, we grab the stoichiometric coefficient $\sigma_{ij}$ corresponding to this metabolite and reaction.

In [6]:
S = let

    # get some data from the model -
    m = model["metabolites"]; # get list of metabolites
    r = model["reactions"]; # get list of reactions
    number_of_rows = length(m); # how many metabolites do we have? (rows)
    number_of_cols = length(r); # how many reactions do we have? (cols)
    S = zeros(number_of_rows,number_of_cols); # initialize an empty stoichiometric matrix

    # let's build a stm -
    for i ∈ eachindex(m)
        metabolite = m[i]["id"]; # we are checking if this metabolite is in the reaction record
        for j ∈ eachindex(r)
            reaction = r[j];
            if (haskey(reaction["metabolites"], metabolite) == true)
                S[i,j] = reaction["metabolites"][metabolite];
            end
        end
    end
    S; 
end;

In [7]:
S

342×469 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

### Covariance matrix $\mathbf{\Sigma}$
The stoichiometric matrix is not square. Thus, we cannot directly compute its eigendecomposition. However, suppose we compute [the covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix) between the columns, i.e., between the reactions stoichiometric vectors. This gives us an idea about the relationship between the network's reactions $i$ and $j$.

In [8]:
Σ = cov(S) # this is col covariance matrix

469×469 Matrix{Float64}:
  0.00292398  -8.57471e-6  -8.57471e-6  …   8.57471e-6   8.57471e-6
 -8.57471e-6   0.00292398  -8.57471e-6      8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6   0.00292398      8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6      8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6      8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6  …   8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6     -0.00292398   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6      8.57471e-6  -0.00292398
 -8.57471e-6  -8.57471e-6  -8.57471e-6      8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6      8.57471e-6   8.57471e-6
 -8.57471e-6  -8.57471e-6  -8.57471e-6  …   8.57471e-6   8.57471e-6
  0.0          0.0          0.0             0.0          0.0
  1.71494e-5   1.71494e-5   1.71494e-5      0.00584795   0.00584795
  ⋮                                     ⋱               
  0.0          0.0          0.0          

__Idea__: Hmmm. The entries of the covariance matrix $\Sigma_{ij} = \sigma_{i}\sigma_{j}\rho_{ij}$ seem a little strange for stoichiometric vectors. Alternatively, suppose we defined another similarity matrix $\hat{\mathbf{\Sigma}}$ using [a Kernel function](https://en.wikipedia.org/wiki/Kernel_method) to compute each entry. The function $\hat{\Sigma}_{ij} = k(\mathbf{\sigma}_{i},\mathbf{\sigma}_{j})$, where $k:\mathbb{R}^{|\mathcal{M}|}\times\mathbb{R}^{|\mathcal{M}|}\rightarrow\mathbb{R}$ is [a Kernel function](https://en.wikipedia.org/wiki/Kernel_method) that takes two columns $\mathbf{\sigma}_{i}$ and $\mathbf{\sigma}_{j}$ for the stoichiometric matrix $\mathbf{S}$ and returns a _similarity score_. 

Let's try this out this idea using the functions exported by [the `KernelFunctions.jl` package](https://github.com/JuliaGaussianProcesses/KernelFunctions.jl) and see what happens.

In [9]:
Σ̂ = let

    # initialize
    (number_of_metabolites, number_of_reactions) = size(S); # dimension of the system
    Σ̂ = zeros(number_of_reactions,number_of_reactions);

    d = SqExponentialKernel(); # what kernel function are we using?
    for i ∈ 1:number_of_reactions
        σᵢ = S[:,i]; # ith reaction
        for j ∈ 1:number_of_reactions
            σⱼ = S[:,j];
            Σ̂[i,j] = d(σᵢ,σⱼ)
        end
    end
    Σ̂
end

469×469 Matrix{Float64}:
 1.0         0.367879    0.367879    …  0.0497871   0.0497871   0.0497871
 0.367879    1.0         0.367879       0.0497871   0.0497871   0.0497871
 0.367879    0.367879    1.0            0.0497871   0.0497871   0.0497871
 0.367879    0.367879    0.367879       0.0497871   0.0497871   0.0497871
 0.367879    0.367879    0.367879       0.0497871   0.0497871   0.0497871
 0.367879    0.367879    0.367879    …  0.0183156   0.0497871   0.0497871
 0.367879    0.367879    0.367879       0.0497871   0.0183156   0.0497871
 0.367879    0.367879    0.367879       0.0497871   0.0497871   0.0183156
 0.367879    0.367879    0.367879       0.0497871   0.0497871   0.0497871
 0.367879    0.367879    0.367879       0.0497871   0.0497871   0.0497871
 0.367879    0.367879    0.367879    …  0.0497871   0.0497871   0.0497871
 0.082085    0.082085    0.082085       0.011109    0.011109    0.011109
 3.39827e-9  3.39827e-9  3.39827e-9     3.39827e-9  3.39827e-9  3.39827e-9
 ⋮           

## Task 2: Compute the eigendecomposition 
In this task, we'll compute the eigendecomposition of the $\mathbf{\Sigma}$ matrix [using the `eigen(...)` method exported by the `LinearAlgebra.jl` package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen). We'll store the eigenvalues in the diagonal $\mathbf{\Lambda}$ matrix, while the eigenvectors will be stored in the $\mathbf{V}$ matrix.
* __Why compute all the values__? In the last task, we will consider what we can do with all the eigenvectors and eigenvalues. Thus, we'll use [the `eigen(...)` method](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) and then grab the last value(s) in task 2 where we look at the largest value, but we'll use the rest later. We'll also check the _magical properties_ of the eigendecomposition of symmetric real matrices; we need all the values for this check.

In [10]:
Λ,V = let

    # initialize -
    A = Σ; # this is the matrix that we will decompose (if we are looking Σ̂, then change)
    (n,m) = size(A); # what is the dimension of A?
    Λ = Matrix{Float64}(1.0*I, n, n); # builds the I matrix, we'll update with λ -
    
    # Decompose using the built-in function
    F = eigen(A);   # eigenvalues and vectors in F of type Eigen
    λ = F.values;   # vector of eigenvalues
    V = F.vectors;  # n x n matrix of eigenvectors, each col is an eigenvector

    # package the eigenvalues into Λ -
    for i ∈ 1:n
        Λ[i,i] = λ[i];
    end

    Λ,V
end;

### Check: Are the magical properties of the eigendecomposition of $\mathbf{\Sigma}$ true?
The covariance matrix $\mathbf{\Sigma}$ is a real-valued, symmetric matrix. So, it's eigenvalues and eigenvectors should have two important properties:
* __Property 1__: All the eigenvalues $\left\{\lambda_{1},\lambda_{2},\dots,\lambda_{m}\right\}$ of the matrix $\mathbf{\Sigma}$ are real-valued.
* __Property 2__: The eigenvectors $\left\{\mathbf{v}_{1},\mathbf{v}_{2},\dots,\mathbf{v}_{m}\right\}$ of the matrix $\mathbf{A}$ are orthogonal, i.e., $\left<\mathbf{v}_{i},\mathbf{v}_{j}\right> = 0$ for $i\neq{j}$. Further, the (normalized) eigenvectors $\hat{\mathbf{v}}_{j} = \mathbf{v}_{j}/\lVert\mathbf{v}_{j}\rVert$ of a symmetric real-valued matrix are orthonormal $\left<\hat{\mathbf{v}}_{i},\hat{\mathbf{v}}_{j}\right> = \delta_{ij}\,\text{for}\,{i,j\in\mathbf{A}}$ where $\delta_{ij}$ is the [Kronecker delta function](https://en.wikipedia.org/wiki/Kronecker_delta).

Let's start by checking __Property 1__. We'll iterate through each eigenvalue and check whether it is real using [the @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) if this test fails, [an `AssertionError` is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [11]:
let
    λ = diag(Λ); # get the elements of the diagonal matrix; this will be the eigenvalues
    for λᵢ ∈ λ
        @assert isreal(λᵢ) == true; # if we fail this test, an AssertionError will be thrown
    end
end

Next, let's check __Property 2__. We'll iterate through each $(i,j)$ pair of eigenvectors and compute the inner product. As it turns out, [the `eigen(...)` method](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) appears to return something very close to [an orthonormal set of eigenvectors](https://en.wikipedia.org/wiki/Orthonormality) for the $\mathbf{\Sigma}$ matrix. Thus, we are checking the condition $\left<\mathbf{v}_{i},\mathbf{v}_{j}\right>=\delta_{ij}$. If this test fails, [an `AssertionError` is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [12]:
let
    # initialize -
    (number_of_rows, number_of_cols) = size(V);

    for i ∈ 1:number_of_rows
        vᵢ = V[:,i]; # get eigenvector i
        for j ∈ 1:number_of_cols
            vⱼ = V[:,j]; # get eigenvector j

            # check logic
            if (i == j)
                @assert dot(vᵢ,vⱼ) ≈ 1.0;
            else
                @assert abs(dot(vᵢ,vⱼ)) ≤ 1e-10 # this is sort of interesting ...
            end
        end
    end
end

## Task 3: What does the largest eigenvalue/eigenvector tell us?
In this task, we'll explore what the largest eigenvector coefficients tell us about the $\mathbf{\Sigma}$ matrix and, by extension, the stoichiometric matrix $\mathbf{S}$. Let's start by getting the eigenvector corresponding to the largest eigenvalue. Save this vector in the `v̂::Array{Float64,1}` variable.
* __What is the `|>` operator doing?__ Functions in Julia can be combined by composing or piping (chaining) them together. The [pipe `|>` operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) enables you to apply a function to the previous function's output. Codes in the class (and most of the stuff I write) use this pattern heavily because I get a strange satisfaction by cramming as much logic as possible on a single line of code.

In [13]:
v̂ = argmax(diag(Λ)) |> i-> V[:,i] # get the largest eigenvector. This is fancy, what is the |> doing?

469-element Vector{Float64}:
 -0.0001379918384368968
 -0.00013759551475015248
 -0.0001375955147768686
 -0.00013799284710265566
  1.614974044265247e-5
  1.614973785206039e-5
  1.6149740436080935e-5
  1.6149740442928607e-5
  1.614974302679023e-5
  1.6149743026790012e-5
  1.6149740436198144e-5
  0.0007011983366113189
 -0.20669155633340722
  ⋮
 -5.57231358427584e-5
 -8.119583495625356e-5
 -0.06753495537233788
 -0.041788603708795176
  0.04486310194343363
 -0.07461340171986781
 -0.04097874036365675
 -0.04097874036363003
 -0.0409783420134054
 -0.04113527318425875
 -0.041135274202159455
 -0.04113527420479559

There are a few ways we can look at the components of the eigenvector.

#### Largest absolute component
First, let's consider taking the absolute value of the elements and then scaling it by the sum of the components. For some vector $\mathbf{z}$, 
the ith scaled component $\sigma(\mathbf{z})_{i}$ is given by:
$$
\sigma(\mathbf{z})_{i} = \frac{\text{abs}(z_{i})}{\sum_{j=1}^{m}\text{abs}(z_{j})}\quad{i=1,2,\dots,m}
$$
The scaled vectors should sum to `1`; thus, we can think about the elements (loosely) as probabilities, i.e., the probability that the ith component is the most important. Let's do the scaling, grab the index of the maximum scaled element [using the `argmax(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.argmax). We'll then pipe `|>` that index into reactions data, and pull out the most important reaction record:

In [14]:
sum(abs.(v̂)) |> T -> (1/T)*abs.(v̂) |> argmax |> i-> model["reactions"][i]

Dict{String, Any} with 9 entries:
  "name"               => "Heme oxygenase 1"
  "metabolites"        => Dict{String, Any}("co_c"=>1.0, "h2o_c"=>3.0, "nadph_c…
  "lower_bound"        => 0.0
  "id"                 => "HOXG"
  "notes"              => Dict{String, Any}("original_bigg_ids"=>Any["HOXG"])
  "gene_reaction_rule" => "Hmox2_AT1 or Hmox1_AT1"
  "upper_bound"        => 1000.0
  "subsystem"          => "Heme Degradation"
  "annotation"         => Dict{String, Any}("bigg.reaction"=>Any["HOXG"], "meta…

#### Softmax
Alternatively, let's use [the `softmax(...)` function](src/Compute.jl) to transform the largest eigenvector in a probability vector (sums to one, all entries are non-negative). The [softmax](https://en.wikipedia.org/wiki/Softmax_function) for some vector $\mathbf{z}$ as
$$
\begin{equation}
\sigma(\mathbf{z})_{i} = \frac{e^{z_{i}}}{\sum_{j=1}^{m}e^{z_{j}}}\quad{i=1,2,\dots,m}
\end{equation}
$$
where $\sigma(\mathbf{z})_{i}$ is the ith components of the transformed eigenvector. We apply [the `argmax(...)` function](https://docs.julialang.org/en/v1/base/collections/#Base.argmax) to the transformed vector to get the largest component

In [15]:
reaction = v̂ |> softmax |> argmax |> i-> model["reactions"][i]

Dict{String, Any} with 9 entries:
  "name"               => "Heme oxygenase 1"
  "metabolites"        => Dict{String, Any}("co_c"=>1.0, "h2o_c"=>3.0, "nadph_c…
  "lower_bound"        => 0.0
  "id"                 => "HOXG"
  "notes"              => Dict{String, Any}("original_bigg_ids"=>Any["HOXG"])
  "gene_reaction_rule" => "Hmox2_AT1 or Hmox1_AT1"
  "upper_bound"        => 1000.0
  "subsystem"          => "Heme Degradation"
  "annotation"         => Dict{String, Any}("bigg.reaction"=>Any["HOXG"], "meta…

### DQ: Hmmm. Does this make sense? 
Let's consider what this result is saying about the network and come to some consensus on whether this makes sense. What happens if we try a different network?

In [16]:
reaction["metabolites"]

Dict{String, Any} with 9 entries:
  "co_c"       => 1.0
  "h2o_c"      => 3.0
  "nadph_c"    => -3.0
  "o2_c"       => -3.0
  "biliverd_c" => 1.0
  "nadp_c"     => 3.0
  "fe2_c"      => 1.0
  "h_c"        => -5.0
  "pheme_c"    => -1.0

In [17]:
connectivity = let

    (number_of_metabolites, number_of_reactions) = size(S);
    connections = Dict{Int,Int}();

    for i ∈ 1:number_of_reactions
        connections[i] = model["reactions"][i] |> d -> d["metabolites"] |> length
    end
    connections
end

Dict{Int64, Int64} with 469 entries:
  56  => 1
  35  => 1
  425 => 5
  429 => 5
  60  => 3
  220 => 5
  308 => 4
  67  => 4
  215 => 2
  73  => 4
  319 => 2
  251 => 3
  115 => 4
  112 => 1
  185 => 1
  348 => 5
  420 => 2
  404 => 5
  365 => 2
  417 => 2
  333 => 9
  86  => 5
  168 => 1
  364 => 4
  207 => 4
  ⋮   => ⋮

In [18]:
i = v̂ |> softmax |> argmax

232

In [19]:
connectivity[232]

9

In [20]:
model["reactions"][333]["metabolites"]

Dict{String, Any} with 9 entries:
  "atp_c"    => -1.0
  "glu__L_c" => 1.0
  "h2o_c"    => -1.0
  "dnad_c"   => -1.0
  "amp_c"    => 1.0
  "nad_c"    => 1.0
  "gln__L_c" => -1.0
  "ppi_c"    => 1.0
  "h_c"      => 1.0